IMPORTS

In [29]:
from google.colab import drive
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import math

LOADING DATA

In [6]:
drive.mount('/content/gdrive')
%cd gdrive/My Drive/Colab Notebooks/IC23/

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/IC23


In [52]:
#! git clone https://github.com/osullik/ic23050.git
#! git pull

In [3]:
orig_data = pd.read_csv("https://raw.githubusercontent.com/osullik/ic23050/main/L4%20Washington%20Fatal%20Crash%20Files%20-%20Washington%20Traffic%20Safety%20Commission/Data_Washington%20Fatal%20Crash%20Survey.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (21,22,23,301,303) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
orig_data.head()

,year,case,par,repjur,crash_dt,crash_tm,accday,accmon,holiday,county,...,nmdistract6,lab,isedtcase,crf1,crf2,crf3,CoRoadName,CoMP,IntCoRoadName,IntCoMP
0,2017,1,E628946,2.0,01/01/2017,2:12,1,1,1.0,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,2,E627989,26.0,01/02/2017,17:14,2,1,1.0,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,4,3747633,263.0,01/01/2017,18:47,1,1,1.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,5,E628691,4.0,01/01/2017,3:50,1,1,1.0,63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,6,3746306,263.0,01/05/2017,9:53,5,1,0.0,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


REVERSE GEOCODING

In [11]:
locator = Nominatim(user_agent="myGeocoder")

In [20]:
# Input: lat and long coordinates
# Output: dictionary containing 'address' -> 'postcode' and other address fields
def reverse_geo(x, y):
  try:
    return locator.reverse((x,y)).raw
  except ValueError:
    print(x, y)

In [41]:
# Input: geo dictionary
# Output: zipcode
def get_zipcode(geodict):
  try:
    return geodict['address']['postcode']
  except KeyError:
    return np.nan

In [13]:
#x, y = 40.7128, -74.0060
#reverse_geo(x, y)

{'place_id': 226179173,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 575213527,
 'lat': '40.71273945',
 'lon': '-74.00593904130275',
 'display_name': 'New York City Hall, 260, Broadway, Lower Manhattan, Manhattan Community Board 1, Manhattan, New York County, City of New York, New York, 10000, United States',
 'address': {'amenity': 'New York City Hall',
  'house_number': '260',
  'road': 'Broadway',
  'quarter': 'Lower Manhattan',
  'neighbourhood': 'Manhattan Community Board 1',
  'suburb': 'Manhattan',
  'county': 'New York County',
  'city': 'City of New York',
  'state': 'New York',
  'ISO3166-2-lvl4': 'US-NY',
  'postcode': '10000',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['40.712445', '40.7130254', '-74.0064455', '-74.0055687']}

In [43]:
orig_data['zipcode'] = orig_data.apply(lambda row: get_zipcode(reverse_geo(row['y'], row['x'])), axis=1)

In [51]:
orig_data.to_csv('/content/gdrive/My Drive/Colab Notebooks/IC23/Data_Washington_Fatal_Crash_Survey_with_zip.csv')

AUGMENTING WITH OSM DATA

In [1]:
# !pip install pyrosm

In [2]:
# from pyrosm import OSM
# from pyrosm import get_data
# from pyrosm.data import sources

In [3]:
# data_with_zip = pd.read_csv('https://raw.githubusercontent.com/osullik/ic23050/main/L4%20Washington%20Fatal%20Crash%20Files%20-%20Washington%20Traffic%20Safety%20Commission/Data_Washington_Fatal_Crash_Survey_with_zip.csv')

In [4]:
# # Grab OSM data from pyrosm
# fp = get_data("washington") # AKA sources.north_america.usa.washington also at '/content/gdrive/My Drive/Colab Notebooks/IC23/washington-latest.osm.bz2'

# # Initialize the OSM parser object
# osm = OSM(fp)

In [ ]:
# # Read all drivable roads
# drive_net = osm.get_network(network_type="driving")
# drive_net.plot()

In [ ]:
# drive_net.head(2)

AUGMENT ZIPCODES WITH DEMOGRAPHICS

In [10]:
# !pip install uszipcode

In [9]:
from uszipcode import SearchEngine  # data.census.gov

/usr/local/lib/python3.8/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [66]:
def augment_zip(zipcode):
  try:
    aug_zip = SearchEngine().by_zipcode(zipcode)
    return [aug_zip.population_density, aug_zip.median_household_income]  # TODO edit fields to be what we want to add to the data listed here: https://uszipcode.readthedocs.io/
  except Exception:
    return [np.nan, np.nan]

In [50]:
# Read in augmented data with zipcodes
orig_data_with_zip = pd.read_csv("https://raw.githubusercontent.com/osullik/ic23050/main/L4%20Washington%20Fatal%20Crash%20Files%20-%20Washington%20Traffic%20Safety%20Commission/Data_Washington_Fatal_Crash_Survey_with_zip.csv")

# Drop all rows with mission zipcode
orig_data_with_zip = orig_data_with_zip[orig_data_with_zip['zipcode'].notna()]  
orig_data_with_zip = orig_data_with_zip.astype({'zipcode':'int'})

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (22,23,24,302,304) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [69]:
orig_data_with_zip['population_density','median_household_income'] = orig_data_with_zip.apply(lambda row: augment_zip(row['zipcode']), axis=1)
orig_data_with_zip[['population_density','median_household_income']] = pd.DataFrame(orig_data_with_zip['population_density','median_household_income'].tolist(),index=orig_data_with_zip.index)
orig_data_with_zip.drop(('population_density','median_household_income'), axis=1)

Exception ignored in: <function SearchEngine.__del__ at 0x7ff631042ca0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/uszipcode/search.py", line 196, in __del__
    self.close()
  File "/usr/local/lib/python3.8/dist-packages/uszipcode/search.py", line 202, in close
    self.ses.close()
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/orm/session.py", line 1811, in close
    self._close_impl(invalidate=False)
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/orm/session.py", line 1853, in _close_impl
    transaction.close(invalidate)
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/orm/session.py", line 923, in close
    transaction.close()
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/engine/base.py", line 2416, in close
    self._do_close()
  File "/usr/local/lib/python3.8/dist-packages/sqlalchemy/engine/base.py", line 2639, in _do_close
    self._close_impl()
  File "/usr/local/lib/python3.8/dist-packages/sqlal

In [72]:
orig_data_with_zip

,Unnamed: 0,year,case,par,repjur,crash_dt,crash_tm,accday,accmon,holiday,...,crf3,CoRoadName,CoMP,IntCoRoadName,IntCoMP,zipcode,population,"(population_density, median_household_income)",population_density,median_household_income
0,0,2017,1,E628946,2.0,01/01/2017,2:12,1,1,1.0,...,NaN,NaN,NaN,NaN,NaN,98201,29582.0,"[3956.0, 43229]",3956.0,43229.0
1,1,2017,2,E627989,26.0,01/02/2017,17:14,2,1,1.0,...,NaN,NaN,NaN,NaN,NaN,98359,4887.0,"[349.0, 71676]",349.0,71676.0
2,2,2017,4,3747633,263.0,01/01/2017,18:47,1,1,1.0,...,NaN,NaN,NaN,NaN,NaN,98103,45911.0,"[9905.0, 75763]",9905.0,75763.0
3,3,2017,5,E628691,4.0,01/01/2017,3:50,1,1,1.0,...,NaN,NaN,NaN,NaN,NaN,99219,NaN,"[None, None]",NaN,NaN
4,4,2017,6,3746306,263.0,01/05/2017,9:53,5,1,0.0,...,NaN,NaN,NaN,NaN,NaN,98174,NaN,"[None, None]",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4127,4127,2021,636,Incid08,334.0,08/08/2021,9:38,8,8,NaN,...,NaN,NaN,NaN,NaN,NaN,99155,1350.0,"[4.0, 25435]",4.0,25435.0
4128,4128,2021,638,Incid09,334.0,09/05/2021,1:36,5,9,NaN,...,NaN,NaN,NaN,NaN,NaN,3971,NaN,"[nan, nan]",NaN,NaN
4129,4129,2021,639,EB48605,263.0,04/21/2021,17:32,21,4,NaN,...,NaN,NaN,NaN,NaN,NaN,98144,26881.0,"[7895.0, 55950]",7895.0,55950.0
4130,4130,2021,639,EB48605,263.0,04/21/2021,17:32,21,4,NaN,...,NaN,NaN,NaN,NaN,NaN,98144,26881.0,"[7895.0, 55950]",7895.0,55950.0


In [77]:
# Drop any columns with more than 1/10 Nan
orig_data_with_zip.dropna(thresh=len(orig_data_with_zip.index)/10, axis=1, inplace=True)

In [79]:
set(orig_data_with_zip.columns)

{('population_density', 'median_household_income'),
 'Unnamed: 0',
 'accday',
 'accmon',
 'adsengaged',
 'adslevel',
 'adspresent',
 'age',
 'agegrp1',
 'agegrp2',
 'aid_inv',
 'ainm_inv',
 'airbag',
 'alcinvol',
 'alcres',
 'alcsts',
 'alctst',
 'bike_fat',
 'bike_inv',
 'body',
 'case',
 'city',
 'city_rd',
 'co_char',
 'commlic',
 'contdev',
 'county',
 'crash_dt',
 'crash_tm',
 'crashtype',
 'criticalevent',
 'criticaleventcat',
 'cty_rd',
 'dayweek',
 'dd_inv',
 'devfunc',
 'diedscene',
 'distract1',
 'dpd_inv',
 'dpnm_inv',
 'dr1617_inv',
 'dr1620_inv',
 'dr1625_inv',
 'dr1820_inv',
 'dr2125_inv',
 'dr65_inv',
 'dr70_inv',
 'dr75_inv',
 'dr_alc7',
 'dr_alc7only',
 'dr_alc8',
 'dr_alc8only',
 'dr_dist',
 'dr_drink',
 'dr_drug',
 'dr_drugonly',
 'dr_fty',
 'dr_imp',
 'dr_spd',
 'dr_unlic',
 'drdist_inv',
 'drf1',
 'drf2',
 'drf3',
 'drf4',
 'drhgt_in',
 'dricond1',
 'driviol1',
 'drivpres',
 'drowsy_inv',
 'druginv',
 'drugres1',
 'drugsts',
 'drugtst1',
 'dzip',
 'ejection',
 'eje